In [20]:
#installing packages in JupyterLab temporarily
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install datetime
!{sys.executable} -m pip install selenium

**Packages**

In [113]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from datetime import datetime
import pandas as pd
import datetime
import getpass
import time
import re

**All pages with urls scraping**

In [122]:
############################################################################
# Downloading offers' urls and titles from the website controlled by Selenium
url = 'https://www.otomoto.pl/osobowe/'

options = webdriver.firefox.options.Options()
options.headless = False

driver = webdriver.Firefox(executable_path=r'C:\Users\Admin\Downloads\geckodriver-v0.29.0-win64\geckodriver.exe')
driver.get(url)
# create BS object from the page source
bs = BS(driver.page_source, 'html.parser')

#### click accept COOKIEs button
# waiting a few seconds to be sure that page will be fully loaded
time.sleep(4)
button = driver.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]').click()

# Extracting number of pages with offers list to iterate
maxSites = bs.find_all('span', {'class':'page'})[-1].text
print("Max site number: " + maxSites)

maxSites = 3 #zmienione do testowania

# All offers' titles and urls scraping
url_list = []
title_list = []

for i in range(int(maxSites)):
    if(i > 0):
        button = driver.find_element_by_xpath('//span[@class="icon-arrow_right"]').click()

    time.sleep(5)
    bs = BS(driver.page_source, 'html.parser')

    titles = bs.find_all('a', {'class':'offer-title__link'}) 

    links = [title['href'] for title in titles]
    
    titles_text = [title.text.strip() for title in titles]
 
    for link in links:
        url_list.append(link)
    for title in titles_text:
        title_list.append(title)
        
    print('loop:  ', i)
    
# Close browser:
driver.quit()

Max site number: 500
loop:   0
loop:   1
loop:   2


In [141]:
print(title_list[1:30])
print("#############")
print(url_list[1])

['Hyundai i20 1.25', 'Peugeot 308 SW', 'Peugeot 3008', 'Peugeot 508 2.0', 'Opel Astra 2.0', 'Volkswagen Golf GTD', 'Fiat 500X 1.4', 'Audi A4 Avant', 'Peugeot 208 1.2', 'Škoda Superb 1.4', 'Škoda Fabia 1.0', 'Škoda Octavia 2.0', 'Renault Captur', 'Kia Sportage 1.6', 'Audi A4 35', 'Volkswagen Polo 1.4', 'Seat Mii', 'Renault Espace', 'Porsche Macan S', 'Audi A7', 'Audi A1', 'Audi A6', 'Audi A5 2.0', 'Audi A8', 'Lexus NX 300h', 'Audi A4', 'Audi A5', 'Audi A4', 'Volkswagen Golf']
#############
https://www.otomoto.pl/oferta/hyundai-i20-model-2015-klima-tempomat-ledy-100-oryg-przebieg-86tys-ID6DFqDw.html#7ce9f71ee3


In [138]:
# import os
# os.chdir(r'C:\Users\Admin\Documents\Repos\MotoScraper')
# cwd = os.getcwd()
# print(cwd)

C:\Users\Admin\Documents\Repos\MotoScraper


In [145]:
titles_encoded = [title.decode("utf-8").encode("windows-1252").decode("utf-8") for title in title_list]
print(titles_encoded)


AttributeError: 'str' object has no attribute 'decode'

In [147]:
# Save urls to df
urls_df = pd.DataFrame({'offer_title': title_list, 'link': url_list})
urls_df.to_csv('urls.csv', index=False, encoding="Windows-1252")

In [110]:
headerOne = {'User-Agent': 'Mozilla/5.0'}

req = Request(site, headers=headerOne)

html = urlopen(req)
bs = BS(html.read(), 'html.parser')